# Convolutional Neural Network dengan PyTorch

### Dataset

Silakan upload cnn_grc.zip ke files.

Alternatif lain, bisa juga dataset di upload ke storage lain lalu nanti didownload dengan cmommand lin

In [ ]:
import torch as pt # turunan
import numpy as np # numerical python
import pandas as pd # bermain dataset
import copy # untuk copy2 an

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/Shareddrives/Deep Learning/cnn.zip" "/content/"

In [ ]:
# extract file zip
# tanda seru artinya ini bukan perintah Python, melainkan perintah command prompt ubuntu
! unzip cnn.zip

Archive:  cnn.zip
replace cnn/BECAK0150.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Packages

In [ ]:
import torch as pt # library utama pytorch
import numpy as np # numerical python
import pandas as pd # bermain dataset
import copy # untuk copy2 an

# libray lain pytorch untuk membangun arsitektur
import torch.nn as nn 
import torch.nn.functional as F

# library dar pytorch untuk pengelolaan dataset khususnya data citra
from torchvision import transforms, datasets, models

### Preprocessing
Pada bagian ini kita akan memperbaiki data agar enak dibaca CNN

In [ ]:
train_dir = "cnn/train/"
test_dir = "cnn/test/"

# fungsi transform bertugas untuk "mengedit" data yang masuk
# macam-macam perintah yang bisa dilakukan bisa cek di sini: https://pytorch.org/docs/stable/torchvision/transforms.html

train_transform = transforms.Compose([ # proses transform dilakukan secara berurutan
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.Resize(500),
    transforms.CenterCrop(224), # 1. di crop ukuran 150x150 di tengah citra
    transforms.ToTensor(), # 2. di ubah ke tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(500),          
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load dataset dengan format folder yang sesuai lalu aplikasikan transformasi ke data yang dibaca
# hasil dari proses ini adalah sebuah list berelemen tuple (tensor_citra, label) disebut dengan objek datasets
train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

In [ ]:
# bagian ini intinya mengubah objek datasets di atas ke format yang lebih baik :)
# insyaAllah akan dijelaskan di pertemuan selanjutnya
trainloaders = pt.utils.data.DataLoader(train_img, batch_size=19, shuffle=True)
testloaders = pt.utils.data.DataLoader(test_img, batch_size=1, shuffle=True)

In [ ]:
# Informasi dari dataset kita
print("train_img type   :",type(train_img)) # objek datasets

print("train_img length :",len(train_img)) # 515 data train
print("test_img length :",len(test_img)) # 66 data test

print("train_img classes:",train_img.classes) # nama kelas
print("train_img classes:",train_img.class_to_idx) # nama kelas

print("train_img[0] type:",type(train_img[0])) # tuple (A, B)
print("train_img[0][0]  :",train_img[0][0].size()) # tensor citra
print("train_img[0][1]  :",train_img[0][1]) # label

# variabel banyak data
len_train = len(train_img)
len_test = len(test_img)

### Buat Arsitektur

In [ ]:
pt.manual_seed(123)
class Net(nn.Module):
   
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        # ukuran sisi = ((ukuran - kernel_size + 2 * padding) / stride + 1)
        # hasil flatten = sisi * sisi * feature_maps
        self.fc1 = nn.Linear(42632, 3) 
    
    def forward(self, x):
        # forward propagation
        x = self.conv1(x) # convolusi
        x = F.relu(x) # fungsi aktivasi
        x = self.pool(x) # pooling
        x = x.reshape(x.shape[0], -1) # flattening
        x = F.relu(self.fc1(x)) # layer biasa
        return x

In [ ]:
modelkita = Net().cuda()

In [ ]:
modelkita

In [ ]:
modelkita = models.resnet18(pretrained=True)

#freezing
for param in modelkita.parameters():
    param.requires_grad = False

# hapus layer terakhir
modelkita.fc = nn.Linear(512, 3)

modelkita = modelkita.cuda()
print(modelkita)

In [ ]:
optimizer = pt.optim.Adagrad(modelkita.parameters(), lr=0.01) # algoritma training
criterion = nn.CrossEntropyLoss() # rumus error

### Mulai Training

In [ ]:
modelkita.train()
for i in range(7): # 10 epoch

    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in trainloaders: # iterasi semua data
        
        image = image.cuda()
        label = label.cuda()

        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1
        
        # backward
        optimizer.zero_grad()
        E.backward()       

        # update
        optimizer.step() 

    print(rata2error/len_train, jumlahbenar/len_train)

### Testing

In [ ]:
modelkita.eval()

with pt.no_grad():
    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in testloaders: # iterasi semua data
        
        image = image.cuda()
        label = label.cuda()

        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1

    print(rata2error/len_test, jumlahbenar/len_test)

### Saving Model

In [ ]:
pt.save(modelkita, "modelkita_v4.pth")

### Inference

In [ ]:
# load model
modelbaru = pt.load("modelkita_v4.pth", map_location=pt.device("cpu"))

In [ ]:
from PIL import Image

modelbaru.eval()

# load single image
single_img = Image.open("/content/cnn/BECAK0150.jpg")

# transform
single_img_transformed = test_transform(single_img)

Y = modelbaru( single_img_transformed.unsqueeze(0) )

prediksi = pt.max(Y, dim=1).indices
print(prediksi)

In [ ]:
prediksi.item()

In [ ]:
from PIL import Image

modelbaru.eval()

# load single image
single_img = Image.open("/content/mobil6.jpg").convert("RGB")

# transform
single_img_transformed = test_transform(single_img)

Y = modelbaru( single_img_transformed.unsqueeze(0) )

prediksi = pt.max(Y, dim=1).indices
print(prediksi)

In [ ]:
prediksi.item()

In [ ]:
from PIL import Image

modelbaru.eval()

# load single image
single_img = Image.open("/content/motor1.jpg").convert('RGB')

# transform
single_img_transformed = test_transform(single_img)

Y = modelbaru( single_img_transformed.unsqueeze(0) )

prediksi = pt.max(Y, dim=1).indices
print(prediksi)

In [ ]:
prediksi.item()